## Notebook for making predictions with deleted constant y samples


In [154]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [156]:
#sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [157]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [158]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

catModel_bc = CatBoostRegressor()

#catModel_abc = CatBoostRegressor()

In [159]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

X_ABC_train = pd.concat([X_A_train, X_BC_train])
Y_ABC_train = pd.concat([Y_A_train, Y_BC_train])

In [160]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

#catModel_bc.fit(X_BC_train,Y_BC_train)

#catModel_abc.fit(X_ABC_train,Y_ABC_train)

Learning rate set to 0.067297
0:	learn: 1245.4675180	total: 44.8ms	remaining: 44.7s
1:	learn: 1183.6100422	total: 50.9ms	remaining: 25.4s
2:	learn: 1126.0666843	total: 58ms	remaining: 19.3s
3:	learn: 1074.2763877	total: 63.2ms	remaining: 15.7s
4:	learn: 1025.8425615	total: 68.3ms	remaining: 13.6s
5:	learn: 981.0020485	total: 73.9ms	remaining: 12.2s
6:	learn: 940.3089146	total: 78.9ms	remaining: 11.2s
7:	learn: 903.9281608	total: 85.4ms	remaining: 10.6s
8:	learn: 869.4601816	total: 91.2ms	remaining: 10s
9:	learn: 838.4679397	total: 96.9ms	remaining: 9.6s
10:	learn: 810.2455315	total: 102ms	remaining: 9.21s
11:	learn: 784.2968710	total: 108ms	remaining: 8.89s
12:	learn: 760.7876381	total: 113ms	remaining: 8.58s
13:	learn: 738.5976552	total: 117ms	remaining: 8.26s
14:	learn: 718.4909591	total: 122ms	remaining: 8.02s
15:	learn: 700.3983799	total: 128ms	remaining: 7.84s
16:	learn: 684.3528899	total: 132ms	remaining: 7.63s
17:	learn: 669.7191760	total: 136ms	remaining: 7.41s
18:	learn: 656.2

In [161]:
# Feature importance
catModel_a.get_feature_importance(prettified=True)

,Feature Id,Importances
0,direct_rad:W,21.934105
1,sum_rad:W,12.519670
2,direct_rad_1h:J,6.088481
3,sun_azimuth:d,5.045104
4,diffuse_rad:W,4.559203
5,wind_speed_u_10m:ms,3.752857
6,day_year,3.683825
7,direct_rad:W_lag_-23,3.147818
8,diffuse_rad_1h:J,2.997938
9,clear_sky_energy_1h:J,2.636594


## Preds

In [162]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

# pred_B = catModel_bc.predict(X_B_test)
# pred_C = catModel_bc.predict(X_C_test)

# pred_A = catModel_abc.predict(X_A_test)
# pred_B = catModel_abc.predict(X_B_test)
# pred_C = catModel_abc.predict(X_C_test)

In [163]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/lagboost_x.csv")

df.shape

(2160, 1)